Metrics:
APY
Yield Volatility (annualized)
Excess vs 3M T-Bill
Information Ratio
For products:
USD: autoUSD, Ethena sUSDe, Aave USDC lending, US Gov short term treasuries
ETH: autoETH, Lido staking (stETH), Aave ETH lending
Time periods: 30 day, 1 year, since inception
*no 1 year on autoUSD since <1 year old
Definitions:
Yield Volatility: stdev of monthly net returns × √12
Excess vs Benchmark: difference between average net APY and the benchmark (3M T-bill for USD; Lido staking rate for ETH).
Information Ratio: average monthly excess return ÷ stdev of monthly excess returns, annuali/zed

# Methods


## Data sources:

Onchain calls on mainnet at the last block of each day in the last year

- autoUSD.converToAssets(1e18) / 1e6 -> USDC value of one autoUSD share
- sUSDe -> chainlink sUSDE -> USD oracle, safe USD value of sUSDe over time
- Aave USDC lending, stataAaveUSDC.convertToAssets(1e6) / 1e6, growth of a share of the aave USDC valut

BIL `SPDR Bloomberg 1-3 Month T-Bill ETF`

https://www.ssga.com/us/en/intermediary/etfs/spdr-bloomberg-1-3-month-t-bill-etf-bil 

> The SPDR® Bloomberg  1-3 Month T-Bill ETF seeks to provide investment results that, before fees and expenses, correspond generally to the price and yield performance of the Bloomberg  1-3 Month U.S. Treasury Bill Index (the "Index")  
> Seeks to provide exposure to publicly issued U.S. Treasury Bills that have a remaining maturities between 1 and 3 months  
> Short duration fixed income is less exposed to fluctuations in interest rates than longer duration securities  
> Rebalanced on the last business day of the month  


As of Oct 26, 2025 ~42.5B AUM

```
!pip install yfinance
import yfinance as yf
tickers = ['BIL']
data = yf.download(tickers, start="2024-10-26", end="2025-10-26", actions=True)
data.to_csv('data.csv')

```

Closing price + cumulative Sum of monthly dividends = simple non compounding return of reinvesting into BIL

note that this does not assume compounding of dividends here, so it is slighly convervative. need to make a better formula here.


# Some Other Notes

- using linear interpolation between for BIL Prices because we don't have prices on the weekends and holidays. 
- BIL sends dividends every 1 month, so the yield is more volitile than the other choices where the yeild is continious. So the volitility of BIL looks a bit higher







In [ ]:
from mainnet_launch.data_fetching.get_state_by_block import get_state_by_one_block, build_blocks_to_use, get_raw_state_by_blocks, safe_normalize_6_with_bool_success, safe_normalize_with_bool_success
from mainnet_launch.constants import ETH_CHAIN, AUTO_USD
import pandas as pd
import plotly .express as px

end_of_day_oct_26_2024_block = 21053236


# https://etherscan.io/address/0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6#readProxyContract
from multicall import Call

# chainlink safe price of sUSDe over time
# https://etherscan.io/address/0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c aave USDC
# https://data.chain.link/feeds/ethereum/mainnet/susde-usd

sUSDe_price_oracle= '0xFF3BC18cCBd5999CE63E788A1c250a88626aD099'
sUSDe = '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497' # mainnet
stata_aave = '0x73edDFa87C71ADdC275c2b9890f5c3a8480bC9E6' # convert to shares calls

aave_convert_to_shares_call = Call(
        stata_aave,
        ["convertToAssets(uint256)(uint256)", int(1e6)],
        [("aave_convert_to_assets",  safe_normalize_6_with_bool_success)],
    )


sUSDe_convert_to_shares_call = Call(
        sUSDe,
        ["convertToAssets(uint256)(uint256)", int(1e18)],
        [("sUSDe_convert_to_assets", safe_normalize_with_bool_success )],
    
    )

autoUSD_nave_per_share_call = Call(
        AUTO_USD.autopool_eth_addr,
        ["convertToAssets(uint256)(uint256)", int(1e18)],
        [("autoUSD_nave_per_share", safe_normalize_6_with_bool_success )],
)



def _process_chainlink_usd_price(success, args):
    """
    Normalize Chainlink latestRoundData -> price as float (answer / 1e6).
    args is: (roundId:uint80, answer:int256, startedAt:uint256, updatedAt:uint256, answeredInRound:uint80)
    """
    round_id, answer, started_at, updated_at, answered_in_round = args
    if success:
        return float(answer) / 1e8


chainlink_sUSDe_usd_price_call = Call(
    sUSDe_price_oracle,
    ["latestRoundData()((uint80,int256,uint256,uint256,uint80))"],
    [("sUSDe_usd_price", _process_chainlink_usd_price)],
)


blocks = build_blocks_to_use(ETH_CHAIN, start_block=end_of_day_oct_26_2024_block)
df = get_raw_state_by_blocks(calls=[
    aave_convert_to_shares_call,    
    autoUSD_nave_per_share_call,
    chainlink_sUSDe_usd_price_call,
], blocks=blocks, chain=ETH_CHAIN) 


bill_df = pd.read_csv('BIL data year to date .csv', parse_dates=['Date'], index_col='Date')[['Dividends', 'Close']]
bill_df['BIL_simple_reinvest_dividends'] = bill_df['Close'] + bill_df['Dividends'].cumsum()
bill_df.index = bill_df.index.date
df.index = df.index.date
large_df = df.merge(bill_df['BIL_simple_reinvest_dividends'], left_index=True, right_index=True, how='left')

large_df['BIL_simple_reinvest_dividends'] = large_df['BIL_simple_reinvest_dividends'].interpolate(method='linear')



In [79]:
first_non_na_values = large_df.apply(lambda col: col.loc[col.first_valid_index()])
norm_one_large_df = large_df / first_non_na_values

recent_df = large_df[large_df.index > pd.to_datetime('2025-04-01').date()]
recent_first_df = recent_df.apply(lambda col: col.loc[col.first_valid_index()])
norm_one_recent_df = recent_df / recent_first_df

px.line(norm_one_large_df, y=['aave_convert_to_assets', 'autoUSD_nave_per_share', 'sUSDe_usd_price', 'BIL_simple_reinvest_dividends'], title='dollar growth since Oct 26, 2024').show()

In [80]:
px.line(norm_one_recent_df, y=['aave_convert_to_assets', 'autoUSD_nave_per_share', 'sUSDe_usd_price', 'BIL_simple_reinvest_dividends'], title='dollar growth since Oct 26, 2024').show()

In [ ]:
# is something strange up here? is my way to estimating the BIL price off this is showing that sUSDe and the treasury bill are very similar and 8% over a year, which seems high?